# INFERENTIAL STATS



In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

# Step 1: Load Excel dataset
excel_path = "/content/drive/MyDrive/CAPSTONE/CAPSTONE_PROJECT/0_dataset/2_CLEANED DATASET/FULLY_CLEANED DATASET.xlsx"   # change to your file path
df = pd.read_excel(excel_path)

# Step 2: Save as CSV
csv_path = "/content/drive/MyDrive/CAPSTONE/CAPSTONE_PROJECT/0_dataset/2_CLEANED DATASET/FULLY_CLEANED DATASET.csv"
df.to_csv(csv_path, index=False)

print("Excel file successfully converted to CSV.")


Excel file successfully converted to CSV.


In [4]:
# Step 3: Load CSV for analysis
df = pd.read_csv(csv_path)

# Quick sanity check
print(df.head())
print(df.info())


/tmp/ipython-input-625198209.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


   Unnamed: 0 Report Number Local Case Number               Agency Name  \
0           0   MCP3126006X         250037402  Montgomery_County_Police   
1           1   MCP2349001B         250037516  Montgomery_County_Police   
2           2   MCP296500BC         250033157  Montgomery_County_Police   
3           3   MCP2159003K         250037509  Montgomery_County_Police   
4           4   MCP312900D6         250034573  Montgomery_County_Police   

        ACRS Report Type      Crash Date/Time            Route Type  \
0           Injury Crash  2025-08-21 17:21:00  Maryland_State_Route   
1  Property Damage Crash  2025-08-22 10:44:00      Interstate_Route   
2  Property Damage Crash  2025-07-25 11:55:00         Bicycle_Route   
3  Property Damage Crash  2025-08-22 10:36:00  Maryland_State_Route   
4  Property Damage Crash  2025-08-03 14:10:00  Maryland_State_Route   

                 Road Name                                  Cross-Street Name  \
0             FREDERICK RD               

# MODEL 1: Injury Severity Prediction (Classification)

## Q1. Feature–Target Association (High Impact)

**Question:**

Is injury severity statistically associated with speed limit, light condition, and collision type?

**Why this matters**

- Validates EDA claim: “Higher speed → higher severity”
- Shows causal plausibility, not just ML accuracy

**Test**

- Chi-Square Test of Independence  
- (Speed limit → binned, others categorical)

In [7]:
import pandas as pd
from scipy.stats import chi2_contingency

# Assume dataframe name is df

# 1. Bin Speed Limit
speed_bins = [0, 25, 35, 45, 55, 100]
speed_labels = ['<=25', '26-35', '36-45', '46-55', '55+']
df['Speed_Bin'] = pd.cut(df['Speed Limit'], bins=speed_bins, labels=speed_labels)

# 2. Define function for Chi-Square test
def chi_square_test(df, feature, target='Injury Severity'):
    contingency_table = pd.crosstab(df[feature], df[target])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    return {
        'Feature': feature,
        'Chi-Square': chi2,
        'p-value': p,
        'Degrees of Freedom': dof
    }

# 3. Run tests
results = []
results.append(chi_square_test(df, 'Speed_Bin'))
results.append(chi_square_test(df, 'Light'))
results.append(chi_square_test(df, 'Collision Type'))

# 4. Results summary
results_df = pd.DataFrame(results)
results_df


,Feature,Chi-Square,p-value,Degrees of Freedom
0,Speed_Bin,2535.236357,0.000000e+00,16
1,Light,217.151658,4.829765e-33,24
2,Collision Type,8196.996965,0.000000e+00,64


### 🔹 Chi-Square Test Results: Injury Severity Associations

The Chi-Square tests show a **statistically significant association** between injury severity and all three features examined: **speed limit**, **light condition**, and **collision type**.

- **Speed Limit (Binned):**  
  The test yields a very large Chi-Square statistic (χ² = 2535.24, *p* < 0.001), indicating that injury severity distributions vary meaningfully across different speed ranges. This strongly supports the claim that **higher speed environments are associated with more severe injuries**, validating the EDA insight with formal statistical evidence.

- **Light Condition:**  
  Light condition is also significantly associated with injury severity (χ² = 217.15, *p* < 0.001). This suggests that **visibility conditions** (e.g., daylight vs. dark) influence crash severity outcomes, reinforcing the role of environmental factors in determining crash impact.

- **Collision Type:**  
  Collision type exhibits the **strongest association** with injury severity (χ² = 8196.99, *p* < 0.001), highlighting that the **nature of the collision** (rear-end, head-on, sideswipe, etc.) is a critical determinant of injury outcomes. This aligns with real-world crash dynamics and adds strong causal plausibility to downstream predictive modeling.

**Overall Conclusion:**  
These results confirm that injury severity is **not independent** of speed, lighting, or collision mechanics. The findings strengthen both the **statistical credibility** and **domain validity** of the capstone analysis, moving beyond surface-level EDA and model accuracy to demonstrate robust inferential support.


## Q2. ANOVA

**Question:**

Does the mean Speed Limit differ significantly across different Injury Severity levels?

**Why this is powerful**

Converts intuition (“higher speed → severe injury”) into statistics

Strong business + policy relevance

Supports your severity model feature importance

**Test Used:**
✔ One-way ANOVA
✔ Numeric vs multi-category target

In [10]:
# Hypotheses

# H₀ (Null Hypothesis): Mean speed limit is the same across all injury severity levels.

# H₁ (Alternative Hypothesis): At least one injury severity level has a different mean speed limit.

In [8]:
import pandas as pd
from scipy.stats import f_oneway

# Assume dataframe name is df

# Group Speed Limit values by Injury Severity
groups = [
    group['Speed Limit'].values
    for _, group in df.groupby('Injury Severity')
]

# One-way ANOVA
f_stat, p_value = f_oneway(*groups)

f_stat, p_value


(np.float64(628.9226730654351), np.float64(0.0))

##  Interpretation (One-Way ANOVA Results)

The one-way ANOVA test indicates a statistically significant difference in the mean **Speed Limit** across different **Injury Severity** levels  
(*F = 628.92, p < 0.001*).

This result provides strong statistical evidence that crashes associated with different injury outcomes occur under systematically different speed environments. In practical terms, it confirms that injury severity is not randomly distributed across speed limits, and that higher-severity injuries are linked to higher posted speed limits.

From a **business and policy perspective**, this finding supports targeted interventions such as speed zoning, enforcement, and roadway design changes in higher-speed corridors to reduce severe injury outcomes.  

From a **modeling standpoint**, it statistically justifies the inclusion and high importance of **Speed Limit** as a predictive feature in injury severity classification models.

Overall, this analysis converts an intuitive safety insight into formal statistical evidence, strengthening both the credibility and defensibility of the capstone project.


# MODEL 2: Driver At Fault Prediction (Classification)

## Q1. Behavioral Association Test

**Question:**

Is driver fault statistically associated with distraction and substance abuse?

**Why**

- Strengthens your behavioral analysis  
- Converts intuition → statistical evidence

**Test**

- Chi-Square Test of Independence


In [ ]:
#  Hypotheses

# For Driver Distraction vs Driver At Fault

# H₀ (Null Hypothesis): Driver fault is independent of driver distraction.

# H₁ (Alternative Hypothesis): Driver fault is associated with driver distraction.

# For Substance Abuse vs Driver At Fault

# H₀ (Null Hypothesis): Driver fault is independent of substance abuse.

# H₁ (Alternative Hypothesis): Driver fault is associated with substance abuse.

In [11]:
import pandas as pd
from scipy.stats import chi2_contingency

# Assume dataframe name is df

def chi_square_test(feature, target='Driver At Fault'):
    contingency = pd.crosstab(df[feature], df[target])
    chi2, p, dof, expected = chi2_contingency(contingency)
    return {
        'Feature': feature,
        'Chi-Square': chi2,
        'p-value': p,
        'Degrees of Freedom': dof
    }

results = []
results.append(chi_square_test('Driver Distracted By'))
results.append(chi_square_test('Driver Substance Abuse'))

results_df = pd.DataFrame(results)
results_df


,Feature,Chi-Square,p-value,Degrees of Freedom
0,Driver Distracted By,50510.336025,0.0,36
1,Driver Substance Abuse,5014.248712,0.0,28


##  Interpretation (Behavioral Association – Chi-Square Results)

The Chi-Square tests reveal strong and statistically significant associations between **Driver At Fault** and both behavioral factors analyzed: **Driver Distraction** and **Substance Abuse**  
(*p < 0.001 for both*).

**Driver Distraction vs Driver At Fault** shows an exceptionally large Chi-Square statistic  
(*χ² = 50,510.34, df = 36*), indicating that fault attribution varies substantially across different types of driver distraction. This result provides compelling statistical evidence that distraction-related behaviors are closely linked to fault responsibility, supporting their role as critical human risk factors in crash outcomes.

**Driver Substance Abuse vs Driver At Fault** also demonstrates a strong association  
(*χ² = 5,014.25, df = 28*), confirming that the presence and type of substance involvement significantly influence whether a driver is deemed at fault. This reinforces well-established road safety findings using formal statistical validation on a large real-world dataset.


## Q2. Two-Sample T-Test

### Question  
Is there a significant difference in mean **Speed Limit** between crashes where the driver was **at fault** versus **not at fault**?

### Why This Works
- **Binary target variable** → ideal for an independent two-sample *t*-test  
- Directly links **road design (speed limits)** to **driver fault**  
- Provides evidence to support **enforcement strategies** and **infrastructure planning decisions**

### Test Used
✔ **Independent two-sample *t*-test**


In [ ]:
#  Hypotheses

# H₀ (Null Hypothesis): Mean speed limit is the same for at-fault and not-at-fault crashes.

# H₁ (Alternative Hypothesis): Mean speed limit differs between at-fault and not-at-fault crashes.

In [12]:
import pandas as pd
from scipy.stats import ttest_ind

# Assume dataframe name is df

# Split Speed Limit by Driver At Fault status
speed_at_fault = df[df['Driver At Fault'] == 'Yes']['Speed Limit']
speed_not_at_fault = df[df['Driver At Fault'] == 'No']['Speed Limit']

# Independent two-sample t-test
t_stat, p_value = ttest_ind(
    speed_at_fault,
    speed_not_at_fault,
    equal_var=False  # Welch's t-test (robust for unequal variances)
)

t_stat, p_value


(np.float64(-44.82409947242886), np.float64(0.0))

## Interpretation (Two-Sample *t*-Test Results)

The independent two-sample *t*-test shows a statistically significant difference in the mean **Speed Limit** between crashes where the driver **was at fault** and **was not at fault**  
(*t = −44.82, p < 0.001*).

The large magnitude of the *t*-statistic indicates that this difference is not only statistically detectable but also systematic across the dataset, rather than being driven by random variation. The negative sign of the *t*-statistic indicates that the mean speed limit is lower for **at-fault crashes** compared to **not-at-fault crashes**, given the ordering of the group subtraction in the test.


# MODEL 3: Vehicle Damage Prediction (Classification)

## Q1. ANOVA

### Question  
Does the mean **Speed Limit** differ significantly across **Vehicle Damage** levels?

### Why This Is Strong
- **Damage severity** is inherently driven by speed, making this relationship theoretically sound  
- Statistically justifies why **speed** is a critical variable for **vehicle damage prediction**  
- Provides excellent analytical support for **insurance risk assessment and pricing use-cases**

### Test Used
✔ **One-way ANOVA**


In [ ]:
#  Hypotheses

# H₀ (Null Hypothesis): Mean speed limit is the same across all vehicle damage levels.

# H₁ (Alternative Hypothesis): At least one vehicle damage level has a different mean speed limit.

In [13]:
import pandas as pd
from scipy.stats import f_oneway

# Assume dataframe name is df

# Group Speed Limit values by Vehicle Damage Extent
groups = [
    group['Speed Limit'].values
    for _, group in df.groupby('Vehicle Damage Extent')
]

# One-way ANOVA
f_stat, p_value = f_oneway(*groups)

f_stat, p_value


(np.float64(1876.089800241534), np.float64(0.0))

## Interpretation (One-Way ANOVA – Vehicle Damage Extent)

The one-way ANOVA test shows a statistically significant difference in the mean **Speed Limit** across different **Vehicle Damage Extent** levels  
(*F = 1876.09, p < 0.001*).

This result provides strong statistical evidence that vehicle damage severity varies systematically across speed environments, confirming that higher posted speed limits are associated with more severe vehicle damage outcomes. The large *F*-statistic indicates that the between-group variation in speed limits across damage categories is substantial compared to within-group variation.

From an **insurance and risk assessment perspective**, this finding supports the use of speed-related features when modeling **claim severity**, **repair costs**, and **premium pricing**, as damage outcomes are clearly influenced by roadway speed conditions.  

From an **analytical standpoint**, it statistically justifies the inclusion of **Speed Limit** as a critical explanatory variable in vehicle damage prediction models.


## Q2. Feature Dependence Test

**Question:**

Is vehicle damage statistically dependent on collision type?

**Why**

- Explains why tree/boosting models outperform linear ones

**Test**

- Chi-Square Test

In [14]:
#  Hypotheses

# H₀ (Null Hypothesis): Vehicle damage extent is independent of collision type.

# H₁ (Alternative Hypothesis): Vehicle damage extent is dependent on collision type.

In [15]:
import pandas as pd
from scipy.stats import chi2_contingency

# Assume dataframe name is df

# Create contingency table
contingency_table = pd.crosstab(
    df['Collision Type'],
    df['Vehicle Damage Extent']
)

# Chi-Square test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

chi2, p_value, dof


(np.float64(34932.17663299716), np.float64(0.0), 96)

##  Interpretation (Chi-Square Test – Vehicle Damage × Collision Type)

The Chi-Square test indicates a highly statistically significant dependence between **Vehicle Damage Extent** and **Collision Type**  
(*χ² = 34,932.18, df = 96, p < 0.001*).

This very large Chi-Square statistic shows that the distribution of vehicle damage levels varies substantially across different collision mechanisms (e.g., rear-end, head-on, sideswipe, single-vehicle). In other words, vehicle damage outcomes are not random with respect to how the collision occurs—they are strongly shaped by collision dynamics.


# MODEL 4: Driver Distraction Cause Prediction (Classification)


## Q1. Environmental Influence on Distraction

**Question:**

Is driver distraction type statistically associated with light and traffic control conditions?

**Why**

- Supports policy & road-design insights  
- Strong interpretability value

**Test**

- Chi-Square Test

In [16]:
#  Hypotheses

# For Light Condition vs Driver Distraction

# H₀ (Null Hypothesis): Driver distraction type is independent of light conditions.

# H₁ (Alternative Hypothesis): Driver distraction type is associated with light conditions.

# For Traffic Control vs Driver Distraction

# H₀ (Null Hypothesis): Driver distraction type is independent of traffic control conditions.

# H₁ (Alternative Hypothesis): Driver distraction type is associated with traffic control conditions.

In [17]:
import pandas as pd
from scipy.stats import chi2_contingency

# Assume dataframe name is df

def chi_square_test(feature, target='Driver Distracted By'):
    contingency = pd.crosstab(df[feature], df[target])
    chi2, p_value, dof, expected = chi2_contingency(contingency)
    return {
        'Feature': feature,
        'Chi-Square': chi2,
        'p-value': p_value,
        'Degrees of Freedom': dof
    }

results = []
results.append(chi_square_test('Light'))
results.append(chi_square_test('Traffic Control'))

results_df = pd.DataFrame(results)
results_df


,Feature,Chi-Square,p-value,Degrees of Freedom
0,Light,466.474585,4.785809e-46,108
1,Traffic Control,1219.358164,1.044889e-114,288


## Interpretation (Environmental Influence on Driver Distraction)

The Chi-Square tests show statistically significant associations between **Driver Distraction Type** and both **Light Conditions** and **Traffic Control Conditions**  
(*p < 0.001 for both*).

**Light Conditions** exhibit a meaningful association with distraction type  
(*χ² = 466.47, df = 108*), indicating that the distribution of distraction behaviors varies across different visibility environments such as daylight, dark-lighted, and dark-unlighted conditions. This suggests that changes in visibility and visual complexity influence the nature of driver distraction.

**Traffic Control Conditions** demonstrate an even stronger association  
(*χ² = 1219.36, df = 288*), highlighting that distraction types differ substantially across intersections, signals, stop controls, and uncontrolled road segments. This reflects how cognitive load and decision complexity introduced by traffic control devices shape driver attention and behavior.


## Q2. ANOVA

### Question  
Does the mean **Crash Hour** differ significantly across different **Driver Distraction** categories?

### Why This Works
- Captures **time-of-day effects** on driver distraction behavior  
- Provides strong **temporal + behavioral insight** into crash dynamics  
- Adds analytical depth to **driver distraction modeling**

### Test Used
✔ **One-way ANOVA**


In [18]:
#  Hypotheses

# H₀ (Null Hypothesis): Mean crash hour is the same across all driver distraction categories.

# H₁ (Alternative Hypothesis): At least one driver distraction category has a different mean crash hour.

In [19]:
import pandas as pd
from scipy.stats import f_oneway

# Assume dataframe name is df

# Group Crash Hour by Driver Distracted By
groups = [
    group['Crash_hour'].values
    for _, group in df.groupby('Driver Distracted By')
]

# One-way ANOVA
f_stat, p_value = f_oneway(*groups)

f_stat, p_value


(np.float64(14.294863472638927), np.float64(2.84862855135289e-44))

##  Interpretation (One-Way ANOVA – Crash Hour × Driver Distraction)

The one-way ANOVA results indicate a statistically significant difference in the mean **Crash Hour** across different **Driver Distraction** categories  
(*F = 14.29, p < 0.001*).

This finding demonstrates that driver distraction behaviors are not uniformly distributed across the day, but instead vary systematically by time. Certain distraction types are more prevalent during specific periods—such as commuting hours, late-night driving, or mid-day travel—reflecting differences in driver workload, alertness, and situational demands.

From a **temporal and behavioral analysis perspective**, this result adds important depth by showing that distraction is influenced not only by environmental and personal factors, but also by **time-of-day effects**.  

From a **modeling standpoint**, it supports the inclusion of **temporal features** (e.g., crash hour, peak vs off-peak indicators) in driver distraction and crash prediction models.


# MODEL 5: Crash Risk Score (Regression)

## Question 1 (ANOVA)

### Question  
Is there a statistically significant difference in the mean **Speed Limit** across different **Collision Types**?

### Test Used
✔ **One-Way ANOVA**


In [ ]:
# Hypotheses

# H₀ (Null Hypothesis): Mean speed limit is the same across all collision types.

# H₁ (Alternative Hypothesis): At least one collision type has a different mean speed limit.

In [22]:
import pandas as pd
from scipy.stats import f_oneway

# Assume dataframe name is df

# Group Speed Limit by Collision Type
groups = [
    group['Speed Limit'].values
    for _, group in df.groupby('Collision Type')
]

# One-way ANOVA
f_stat, p_value = f_oneway(*groups)

f_stat, p_value


(np.float64(1571.5629260674748), np.float64(0.0))

## Interpretation (One-Way ANOVA – Speed Limit × Collision Type)

The one-way ANOVA results indicate a statistically significant difference in the mean **Speed Limit** across different **Collision Types**  
(*F = 1571.56, p < 0.001*).

This finding demonstrates that collision mechanisms are systematically associated with different speed environments, rather than occurring uniformly across all road types. High-energy collision types (such as head-on or high-speed rear-end crashes) tend to occur on higher-speed roadways, while lower-energy collision types (such as backing or low-speed sideswipes) are more prevalent in lower-speed environments.

From a **crash dynamics and road design perspective**, this result supports differentiated safety strategies based on collision context—such as enhanced median protection on higher-speed corridors and improved access management in lower-speed areas.  

From a **modeling standpoint**, it justifies treating **Collision Type** and **Speed Limit** as interrelated but distinct explanatory variables, strengthening feature interactions in predictive models.


## Question 2

### Question  
Is there a statistically significant difference in the mean **Speed Limit** across different **Route Types**?

### Test Used
✔ **One-Way ANOVA**


In [ ]:
#  Hypotheses

# H₀ (Null Hypothesis): Mean speed limit is the same across all route types.

# H₁ (Alternative Hypothesis): At least one route type has a different mean speed limit.

In [23]:
import pandas as pd
from scipy.stats import f_oneway

# Assume dataframe name is df

# Group Speed Limit by Route Type
groups = [
    group['Speed Limit'].values
    for _, group in df.groupby('Route Type')
]

# One-way ANOVA
f_stat, p_value = f_oneway(*groups)

f_stat, p_value


(np.float64(1739.894732120688), np.float64(0.0))

##  Interpretation (One-Way ANOVA – Speed Limit × Route Type)

The one-way ANOVA results show a statistically significant difference in the mean **Speed Limit** across different **Route Types**  
(*F = 1739.89, p < 0.001*).

This finding confirms that route classifications (such as interstate routes, state highways, county roads, and local roadways) are associated with distinct speed environments, reflecting intentional roadway design and functional hierarchy. Higher-class routes are systematically designed to support higher speeds, while local and specialized routes operate under lower speed limits.

From a **transport planning and policy perspective**, this result provides strong statistical justification for **route-based speed zoning**, **enforcement strategies**, and **safety interventions** tailored to roadway function.  

From a **modeling and analytical standpoint**, it reinforces the importance of including **Route Type** as a key contextual feature when analyzing **crash risk**, **severity**, or **driver behavior**.


| Model Name | Test Name | Hypothesis | Results (Statistic, p-value, Dof/F) |
|---|---|---|---|
| **Injury Severity Prediction** | Chi-Square Test of Independence (Speed Limit vs. Injury Severity) | H₀: Speed is independent of Injury Severity. | χ² = 2535.24, p < 0.001, Dof = 16 |
| | Chi-Square Test of Independence (Light vs. Injury Severity) | H₀: Light is independent of Injury Severity. | χ² = 217.15, p < 0.001, Dof = 24 |
| | Chi-Square Test of Independence (Collision Type vs. Injury Severity) | H₀: Collision Type is independent of Injury Severity. | χ² = 8196.99, p < 0.001, Dof = 64 |
| | One-way ANOVA (Speed Limit by Injury Severity) | H₀: Mean Speed Limit is same across all Injury Severity levels. | F = 628.92, p < 0.001 |
| **Driver At Fault Prediction** | Chi-Square Test of Independence (Distraction vs. Driver At Fault) | H₀: Driver fault is independent of distraction. | χ² = 50510.34, p < 0.001, Dof = 36 |
| | Chi-Square Test of Independence (Substance Abuse vs. Driver At Fault) | H₀: Driver fault is independent of substance abuse. | χ² = 5014.25, p < 0.001, Dof = 28 |
| | Two-Sample t-Test (Speed Limit by Driver At Fault) | H₀: Mean Speed Limit is same for at-fault/not-at-fault crashes. | t = −44.82, p < 0.001 |
| **Vehicle Damage Prediction** | One-way ANOVA (Speed Limit by Vehicle Damage Extent) | H₀: Mean Speed Limit is same across all Vehicle Damage Extent levels. | F = 1876.09, p < 0.001 |
| | Chi-Square Test (Collision Type vs. Vehicle Damage Extent) | H₀: Vehicle Damage Extent is independent of Collision Type. | χ² = 34932.18, p < 0.001, Dof = 96 |
| **Driver Distraction Cause Prediction** | Chi-Square Test (Light vs. Driver Distracted By) | H₀: Distraction type is independent of light conditions. | χ² = 466.47, p < 0.001, Dof = 108 |
| | Chi-Square Test (Traffic Control vs. Driver Distracted By) | H₀: Distraction type is independent of traffic control conditions. | χ² = 1219.36, p < 0.001, Dof = 288 |
| | One-way ANOVA (Crash Hour by Driver Distracted By) | H₀: Mean Crash Hour is same across all Driver Distraction categories. | F = 14.29, p < 0.001 |
| **Crash Risk Score** | One-way ANOVA (Speed Limit by Collision Type) | H₀: Mean Speed Limit is same across all Collision Types. | F = 1571.56, p < 0.001 |
| | One-way ANOVA (Speed Limit by Route Type) | H₀: Mean Speed Limit is same across all Route Types. | F = 1739.89, p < 0.001 |